# Get resources from bio.tools

See the [bio.tools API reference](https://biotools.readthedocs.io/en/latest/api_reference.html) and [API usage guide](https://biotools.readthedocs.io/en/latest/api_usage_guide.html) on how to query [bio.tools](https://bio.tools/) records.

JSON schema for bio.tools: https://github.com/bio-tools/biotoolsSchema/blob/master/jsonschema/biotoolsj.json

In [ ]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent = 2)

# bio.tools REST API
base_url = "https://bio.tools/api"
sign_in_url = f"{base_url}/rest-auth/login/" # for submitting resources

In [ ]:
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

url = f"{base_url}/tool/"

In [ ]:
# search for rources from BBMRI collection
params = {
    #"q": ["multi-omics"],
    "collectionID": "BBMRI" # "Rare Disease", "RD-connect"
}
response = requests.request("GET", url, headers = headers, params = params)

# structure of result
print(response.json().keys())

# number of results
print(response.json()["count"])

# print 1st result
pp.pprint(response.json()["list"][0])

In [ ]:
# resources from rare disease collection
params = {
    "collectionID": "Rare Disease"
}
response = requests.request("GET", url, headers = headers, params = params)
print(response.json()["count"])

In [ ]:
# use a general search term 
params = {
    "q": "translational medicine"
}
response = requests.request("GET", url, headers = headers, params = params)
print(response.json()["count"])

In [ ]:
# query collection EUCAIM, open access only
params = {
    "accessibility": "Open access",
    "collectionID": "EUCAIM"
}
response = requests.request("GET", url, headers = headers, params = params)
print(response.json()["count"])

In [ ]:
# summarise previous results - record name and license
print(response.json()["list"][0].keys())
pp.pprint([(record["name"], record["license"]) for record in response.json()["list"]])

In [ ]:
params = {
    "q": "interoperability"
    #"collectionID": "EUCAIM"
}
response = requests.request("GET", url, headers = headers, params = params)
print(response.json()["count"])

In [ ]:
pp.pprint([(record["name"]) for record in response.json()["list"]])